## Microtubule Dynamics ABM Julia notebook
This notebook is intended as a development and testing file , so i will change regulary.

nächstes meeting 05.10 10:45

### _TODO_ ::

    - Drei Abbildungen erstellen für klein 100, mittel300, groß1000 ,a 50 mal begrenzte randbedingungen ✓
    - seed funktion?!
    - Standartdabweichung (korridor) 
    - Rohdaten gut aufbewahren, sinnvolle beschriftungen  ✓
    - Periodischen Randbedingugen mit in microtubuli modell implementieren ✓

  - Wachstumsgeswindigkeit (erstmal ohne Depolymerisation) ~ N.Agenten/Gridsize & P-polymerisiert
  
  


In [1]:
using Agents        # for ABM
using CairoMakie    # Plotting 
using Distributions # binomial
using StatsBase: counts    
using DataFrames    # ?
using Statistics: mean
using InteractiveDynamics # plotting
using GLMakie       # 
using DrWatson: @dict #  model params, plotting
include("Polymerize.jl")
#include("agent_group_cont.jl")

depolymerize (generic function with 1 method)

In [2]:
#=  Agent representing 1 tubulin dimer
 =#

@agent tubulin GridAgent{2} begin
    polym::Int64
    GDP::Bool
    # Plus::Int64
end

In [3]:
#= Main Function to initialize the model=#

function initialize(;Nstarts, numagents, periodic, griddims, p_hyd,  p_polym,
                        p_depolym_GTP,p_depolym_GDP)
   
    P_hyd = Binomial(1,p_hyd)                      # Prob to hydrolize from GTP to GDP: 
    P_polym = Binomial(1,p_polym)                  # Prob to Polymerize/bind to the microtuble 
    P_depolym_GTP = Binomial(1,1-p_depolym_GTP)    # Prob to depolimerize if not hydrolized (GTP - tubulin) = lower
    P_depolym_GDP = Binomial(1,1-p_depolym_GDP)    # Prob to depolimerize if hydrolized (GDP - tubulin) = higher

    
 properties = @dict griddims numagents Nstarts p_depolym_GTP p_depolym_GDP p_polym p_hyd P_hyd P_polym P_depolym_GTP P_depolym_GDP
 
    properties[:tick] = 0
   # properties[:MTmL] = Vector{Float64}()
   # properties[:MTstdL] = Vector{Float64}()
    space = GridSpace(griddims, periodic = periodic ) # initialize Gridspace
    
    model = ABM(tubulin, space;                     # create model
        scheduler = Schedulers.randomly,
        properties = properties )
    
    id = 0                                  # populize Space ->  
    for _ in 1:Nstarts # Startingpoints:
        id += 1
        agent =  tubulin(id,griddims./2,id,true)
        if id == 1
            add_agent_pos!(agent,model)
        else
            add_agent_single!(agent,model) # add additional startingpoints to random positions 
        end
    end
    for _ in 1:numagents 
        id += 1
        agent =  tubulin(id,(1,1),0,false)
        add_agent_single!(agent,model) # add agents to random positions
    end
    return model
end

initialize (generic function with 1 method)

In [4]:
# Agent Step function

function agent_step!(agent,model)
if agent.id > model.Nstarts
    if agent.polym == 0   # Logic for unpolymerized agents:    
            walk!(agent, rand, model, ifempty =true)
            polymerize(agent,model) 
        end
    if agent.polym > 0 # Logic for polymerized agents:    
        if agent.GDP == false   # if not hydrolized
            agent.GDP = rand(model.P_hyd,1)[1] # hydrolize with P_hyd
            end
           depolymerize(agent,model)
        end
     end
end

agent_step! (generic function with 1 method)

In [5]:
# model-Step func:
function model_step!(model)
   # model.MTmL = mean(counts([model.agents[i].polym for i in 1:length(model.agents)])[setdiff(1:end, 1), 1])
   # model.MTstdL= std(counts([model.agents[i].polym for i in 1:length(model.agents)])[setdiff(1:end, 1), 1])
    model.tick += 1
    model.P_hyd = Binomial(1,model.p_hyd)                      # Prob to hydrolize from GTP to GDP: 
    model.P_polym = Binomial(1,model.p_polym)                  # Prob to Polymerize/bind to the microtuble 
    model.P_depolym_GTP = Binomial(1,1-model.p_depolym_GTP)    # Prob to depolimerize if not hydrolized (GTP - tubulin) = lower
    model.P_depolym_GDP = Binomial(1,1-model.p_depolym_GDP)    # Prob to depolimerize if hydrolized (GDP - tubulin) = higher
end

model_step! (generic function with 1 method)

In [ ]:
step!(model,agent_step!,model_step! ,1)

In [13]:
# initialization:

model = initialize(
    griddims =(100,100),
    periodic= true,
    Nstarts=4,
    numagents = 1000,
    p_polym=0.80,
    p_hyd= 0.02,
    p_depolym_GTP=0,
    p_depolym_GDP=0.03)

#step!(model,agent_step!,model_step! ,1, agents_first =true)

AgentBasedModel with 1004 agents of type tubulin
 space: GridSpace with size (100, 100), metric=chebyshev, periodic=true
 scheduler: randomly
 properties: Dict{Symbol, Any}(:tick => 0, :griddims => (100, 100), :P_hyd => Binomial{Float64}(n=1, p=0.02), :p_depolym_GDP => 0.03, :Nstarts => 4, :P_depolym_GTP => Binomial{Float64}(n=1, p=1.0), :P_depolym_GDP => Binomial{Float64}(n=1, p=0.97), :p_polym => 0.8, :p_hyd => 0.02, :p_depolym_GTP => 0…)

In [14]:
# Interactive Model
using InteractiveDynamics
using GLMakie

params = Dict(
    :p_hyd => 0:0.01:1,
    :p_polym => 0:0.01:1,
    :p_depolym_GTP => 0:0.01:1,
    :p_depolym_GDP =>0:0.01:1
)
GLMakie.activate!()
# stats
# coloring
groupcolor(a) = a.polym >= 1 ? :red : :blue
groupmarker(a) = a.GDP == true ? :circle : :octagon
mdata=[mean_MT_size,sd_MT_size]
mlabels=["mean MT length", "sd"]
adata = [(:GDP,sum)]
alabels = ["Hydrolized"]

fig, adf, mdf = abm_data_exploration(
    model, agent_step!, model_step!, params;
    ac=groupcolor, am=groupmarker, adata,alabels,mdata,mlabels
)

(Scene (1600px, 800px):
  102 Plots:
    ├ Combined{Makie.poly, Tuple{Vector{Vector{Point{2, Float32}}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{Vector{Tuple{AbstractString, Point{2, Float32}}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ LineSegments{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{Vector{Tuple{AbstractString, Point{2, Float32}}}}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ Lines{Tuple{Vector{Point{2, Float32}}}}
    ├ MakieCore.Text{Tuple{String}}
    ├ Scatter{Tuple{Ve

In [ ]:
step!(model,agent_step!,model_step!,1)

In [7]:
# Function to calculate mean of microtubule length
function mean_MT_size(model)
    meanMTL=mean(counts([model.agents[i].polym for i in 1:length(model.agents)])[setdiff(1:end, 1), 1])

end

mean_MT_size (generic function with 1 method)

In [8]:
# Function to calculate standart devation of microtubule length
function sd_MT_size(model)
    stdMTL=std(counts([model.agents[i].polym for i in 1:length(model.agents)])[setdiff(1:end, 1), 1])
end

sd_MT_size (generic function with 1 method)

In [11]:
### Data collection
#using Statistics: mean
adata = [:pos,:GDP,:polym,] # agent data
mdata=[:tick, mean_MT_size,sd_MT_size]
#data = run!(model,agent_step!,100;adata)

data, _ = run!(model,agent_step!,100;adata, mdata)



(101404×5 DataFrame
    Row │ step   id     pos        GDP    polym 
        │ Int64  Int64  Tuple…     Bool   Int64 
────────┼───────────────────────────────────────
      1 │     0      1  (50, 50)    true      1
      2 │     0      2  (85, 87)    true      2
      3 │     0      3  (34, 42)    true      3
      4 │     0      4  (94, 90)    true      4
      5 │     0      5  (38, 100)  false      0
      6 │     0      6  (44, 18)   false      0
      7 │     0      7  (18, 65)   false      0
      8 │     0      8  (85, 33)   false      0
      9 │     0      9  (27, 24)   false      0
     10 │     0     10  (64, 17)   false      0
     11 │     0     11  (84, 79)   false      0
   ⋮    │   ⋮      ⋮        ⋮        ⋮      ⋮
 101395 │   100    995  (48, 41)   false      0
 101396 │   100    996  (1, 7)     false      0
 101397 │   100    997  (49, 59)   false      0
 101398 │   100    998  (76, 84)   false      0
 101399 │   100    999  (25, 63)   false      0
 101400 │   100   1

In [ ]:
parameters=Dict(
            :griddims => (100,100),
            :periodic => true,
            :Nstarts => collect(3:6),
            :numagents => [800,1000,1200],
            :p_hyd => [0.001,0.005,0.01,0.05],
            :p_polym=>[0.7,0.8,0.9],
            :p_depolym_GTP=> [0.005,0.001],
            :p_depolym_GDP=>[0.05,0.01,0.1])


In [ ]:
data = paramscan(parameters, initialize; adata, mdata, agent_step!, model_step!, n = 3)

In [ ]:
## Ensemble run RW
# Build ensemble of models

models = [initialize(; Nstarts=0,
        periodic= true,
        numagents = 1000,
        griddims=(x,x), 
        p_hyd=0,
        p_polym=0,
        p_depolym_GTP=0,
        p_depolym_GDP=0) for x in repeat([80, 100, 200], 30)];

adata = [:pos]

adf, = ensemblerun!(models, agent_step!, model_step!, 2000; adata)
using CSV
CSV.write("data/Periodic_ensemble_80_100_200_1kagents_a30_2500_step_runs.csv", adf)

In [ ]:
## Ensemble run Microtubuli simulation
models = [initialize(; Nstarts=15,
        periodic= true,
        numagents = 1000,
        griddims=(x,x), 
        p_polym=0.80,
        p_hyd= 0.02,
        p_depolym_GTP=0,
        p_depolym_GDP=0.03) for x in repeat([120,150, 200], 30)];

adata = [:pos]

adf, = ensemblerun!(models, agent_step!, model_step!, 2000; adata)

In [ ]:
?paramscan


In [ ]:
## MSD for multiple ensembles

MSDS = DataFrame(Step= levels(adf.step))
for i in levels(adf.ensemble)
    name = Symbol("ens_$i")
    MSDS[!,name] = MSD(adf[(adf.ensemble .== i ),:])
end

CSV.write("Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv", MSDS)

In [ ]:
CSV.write("Periodic_simulation_120_150_200_1kagents_a30_2000_step_runs.csv", adf)